# Black–Scholes Pricing + Discrete-Time Delta Hedging (Illustration)

**Notebook:** `01_bs_delta_hedging.ipynb`  
**Repo:** `black-scholes-hedging`

This notebook:
- Uses Black–Scholes to price a European call and compute its delta.
- Illustrates **discrete-time delta hedging** along a historical price path.
- Reports hedging error (because hedging is discrete + inputs are estimated).

Not trading advice. This is an educational, documented illustration.

## 0) Setup

We import our own Black–Scholes functions from `src/black_scholes.py`.

If you run this notebook in Colab/Jupyter, ensure the repo root is the working directory.

In [ ]:
import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Make plots readable
plt.rcParams["figure.dpi"] = 120

# --- paths ---
ROOT = Path(".").resolve()
DATA_RAW = ROOT / "data" / "raw"
DATA_PROCESSED = ROOT / "data" / "processed"
FIGURES = ROOT / "figures"

for p in [DATA_RAW, DATA_PROCESSED, FIGURES]:
    p.mkdir(parents=True, exist_ok=True)

# --- import local module ---
sys.path.append(str(ROOT / "src"))
from black_scholes import bs_call_price, bs_call_delta

print("OK")
print("ROOT:", ROOT)
print("Raw data folder:", DATA_RAW)
print("Processed data folder:", DATA_PROCESSED)
print("Figures folder:", FIGURES)

## 1) Get historical price data

We use **Stooq** (free CSV via URL) to keep this lightweight and reproducible.

- Underlying: `SPY` (S&P 500 ETF)
- Frequency: daily

If the download fails in your environment, you can manually download the CSV and place it into `data/raw/`.

In [ ]:
symbol = "spy.us"  # Stooq uses lowercase like spy.us
url = f"https://stooq.com/q/d/l/?s={symbol}&i=d"
raw_path = DATA_RAW / f"{symbol}_daily.csv"

df = pd.read_csv(url)
df.to_csv(raw_path, index=False)

df.head(), raw_path

In [ ]:
# Clean + keep only what we need
df["Date"] = pd.to_datetime(df["Date"])
df = df.sort_values("Date").reset_index(drop=True)

# Use Close as spot proxy
df = df[["Date", "Close"]].rename(columns={"Close": "S"})
df = df.dropna().reset_index(drop=True)

print("Rows:", len(df))
df.tail()

## 2) Returns + volatility estimate

Black–Scholes uses a constant volatility parameter `sigma`.

Here we estimate sigma from historical log returns using a rolling window.

**Assumptions (illustrative):**
- 252 trading days per year
- Rolling window: 60 days
- Volatility estimate: annualized std of daily log returns

In [ ]:
TRADING_DAYS = 252
VOL_WINDOW = 60

df["log_ret"] = np.log(df["S"]).diff()
df["sigma"] = (
    df["log_ret"].rolling(VOL_WINDOW).std() * np.sqrt(TRADING_DAYS)
)

df[["Date", "S", "log_ret", "sigma"]].tail(10)

In [ ]:
# Plot the spot and the sigma estimate
fig, ax = plt.subplots(figsize=(9, 4))
ax.plot(df["Date"], df["S"])
ax.set_title("Underlying spot (Close)")
ax.set_xlabel("Date")
ax.set_ylabel("S")
fig.tight_layout()
fig.savefig(FIGURES / "spot_series.png", bbox_inches="tight")
plt.show()

fig, ax = plt.subplots(figsize=(9, 4))
ax.plot(df["Date"], df["sigma"])
ax.set_title(f"Rolling volatility estimate (window={VOL_WINDOW})")
ax.set_xlabel("Date")
ax.set_ylabel("sigma (annualized)")
fig.tight_layout()
fig.savefig(FIGURES / "rolling_sigma.png", bbox_inches="tight")
plt.show()

## 3) Define an option to hedge

We choose a simple illustrative option:
- European call
- Strike `K` set to spot at hedge start (ATM)
- Maturity `T` = 30 calendar days ≈ 30/365 years
- Constant risk-free rate `r` (illustrative)

We then:
- Recompute BS price and delta each day using current `S`, time-to-maturity `tau`, and rolling `sigma`.
- Run a **discrete delta-hedging** strategy (daily rebalancing).

In [ ]:
# Choose a period where sigma is available
df_h = df.dropna(subset=["sigma"]).reset_index(drop=True)

# Hedge horizon
N_DAYS = 30                 # number of trading days to hedge
r = 0.03                    # illustrative risk-free rate (annual)
DAY_COUNT = 365.0           # for tau

# Pick a start index far enough from the end
start_idx = len(df_h) - 400  # arbitrary; change if you want a different window
end_idx = start_idx + N_DAYS

path = df_h.loc[start_idx:end_idx, ["Date", "S", "sigma"]].copy().reset_index(drop=True)

S0 = float(path.loc[0, "S"])
K = S0  # ATM at inception

path.head(), (S0, K, r, N_DAYS)

In [ ]:
# Time-to-maturity tau for each step (in years)
# We treat each row as one trading day step, but keep a simple calendar conversion.
path["t"] = np.arange(len(path))
path["tau"] = (N_DAYS - path["t"]) / DAY_COUNT

# BS price and delta per day
path["C"] = path.apply(lambda row: bs_call_price(row["S"], K, r, row["sigma"], row["tau"]), axis=1)
path["delta"] = path.apply(lambda row: bs_call_delta(row["S"], K, r, row["sigma"], row["tau"]), axis=1)

path.head()

## 4) Discrete-time delta hedging simulation

We simulate hedging a **short 1 call** position.

At inception (t=0):
- Sell the call for `C0`.
- Buy `delta0` shares.
- Put remaining cash into a bank account accruing at rate `r`.

Each day:
- Accrue interest on cash.
- Rebalance stock position to match new delta.

At maturity:
- Option payoff is `max(S_T - K, 0)`.
- Portfolio value minus payoff = hedging error.

This will generally not be 0 because:
- hedging is discrete,
- sigma is estimated and time-varying,
- BS assumptions don’t hold exactly.

In [ ]:
dt = 1.0 / TRADING_DAYS

# Initialize
C0 = float(path.loc[0, "C"])
delta0 = float(path.loc[0, "delta"])
S0 = float(path.loc[0, "S"])

# Short call -> receive premium
# Use premium to buy delta shares, remainder is cash
shares = delta0
cash = C0 - shares * S0

rows = []
rows.append({
    "t": 0,
    "Date": path.loc[0, "Date"],
    "S": S0,
    "C": C0,
    "delta": delta0,
    "shares": shares,
    "cash": cash,
    "portfolio": shares * S0 + cash,
})

# Rebalance through time
for i in range(1, len(path)):
    S = float(path.loc[i, "S"])
    delta = float(path.loc[i, "delta"])
    C = float(path.loc[i, "C"])

    # accrue interest on cash
    cash = cash * np.exp(r * dt)

    # rebalance shares to new delta
    target_shares = delta
    trade = target_shares - shares
    cash = cash - trade * S
    shares = target_shares

    portfolio = shares * S + cash
    rows.append({
        "t": i,
        "Date": path.loc[i, "Date"],
        "S": S,
        "C": C,
        "delta": delta,
        "shares": shares,
        "cash": cash,
        "portfolio": portfolio,
    })

hedge = pd.DataFrame(rows)
hedge.tail()

In [ ]:
# Maturity payoff and hedging error
S_T = float(hedge.loc[len(hedge)-1, "S"])
payoff = max(S_T - K, 0.0)
final_portfolio = float(hedge.loc[len(hedge)-1, "portfolio"])

# Since we are short the call, we owe payoff
hedging_error = final_portfolio - payoff

println = lambda *x: print(*x)
println("K:", K)
println("S_T:", S_T)
println("Payoff (short call owes):", payoff)
println("Final hedging portfolio:", final_portfolio)
println("Hedging error (portfolio - payoff):", hedging_error)

## 5) Visualize the hedge

- Consider the option price vs portfolio value.
- Track delta through time.
- Track cash and shares.

In [ ]:
fig, ax = plt.subplots(figsize=(9, 4))
ax.plot(hedge["Date"], hedge["portfolio"], label="Hedge portfolio")
ax.plot(hedge["Date"], hedge["C"], label="BS call price")
ax.set_title("Hedge portfolio value vs option price")
ax.set_xlabel("Date")
ax.set_ylabel("Value")
ax.legend()
fig.tight_layout()
fig.savefig(FIGURES / "portfolio_vs_call.png", bbox_inches="tight")
plt.show()

fig, ax = plt.subplots(figsize=(9, 4))
ax.plot(hedge["Date"], hedge["delta"])
ax.set_title("Delta through time")
ax.set_xlabel("Date")
ax.set_ylabel("Delta")
fig.tight_layout()
fig.savefig(FIGURES / "delta_series.png", bbox_inches="tight")
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(9, 4))
ax.plot(hedge["Date"], hedge["cash"], label="Cash")
ax.set_title("Cash account (with interest accrual)")
ax.set_xlabel("Date")
ax.set_ylabel("Cash")
ax.legend()
fig.tight_layout()
fig.savefig(FIGURES / "cash_series.png", bbox_inches="tight")
plt.show()

fig, ax = plt.subplots(figsize=(9, 4))
ax.plot(hedge["Date"], hedge["shares"], label="Shares held")
ax.set_title("Shares held (delta hedge)")
ax.set_xlabel("Date")
ax.set_ylabel("Shares")
ax.legend()
fig.tight_layout()
fig.savefig(FIGURES / "shares_series.png", bbox_inches="tight")
plt.show()

## 6) Save processed outputs

We store:
- the path used (`data/processed/...`)
- the hedging simulation table (`data/processed/...`)

So the repo has reproducible artifacts even if someone doesn’t run the notebook.

In [ ]:
processed_path = DATA_PROCESSED / "hedge_path.csv"
hedge_path = path.copy()
hedge_path.to_csv(processed_path, index=False)

processed_hedge = DATA_PROCESSED / "hedge_simulation.csv"
hedge.to_csv(processed_hedge, index=False)

processed_path, processed_hedge

## 7) Minimal results summary (for README/report)

Below is a small text summary you can copy into `reports/` or the README later.

In [ ]:
summary = {
    "symbol": symbol,
    "start_date": str(hedge.loc[0, "Date"].date()),
    "end_date": str(hedge.loc[len(hedge)-1, "Date"].date()),
    "K": float(K),
    "r": float(r),
    "VOL_WINDOW": int(VOL_WINDOW),
    "N_DAYS": int(N_DAYS),
    "C0": float(C0),
    "S_T": float(S_T),
    "payoff": float(payoff),
    "final_portfolio": float(final_portfolio),
    "hedging_error": float(hedging_error),
}

pd.Series(summary)

### Next

1) Run this notebook locally/Colab and commit generated `figures/` and `data/processed/` outputs.  
2) Add a short `reports/summary.md` with the assumptions and the single-run outcome.  
3) Optional: repeat hedging over many rolling windows to show a distribution of hedging errors.
